In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from tensorflow import keras
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

#Reading in our training data
df = pd.read_csv("./kaggle/chessData.csv")
#Reading in our validation data
Edf = pd.read_csv("./kaggle/random_evals.csv")

In [10]:
df.head(10)

,FEN,Evaluation
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,-10
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,+56
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,-9
3,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...,+52
4,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...,-26
5,rnbqkb1r/ppp2ppp/4pn2/3p4/3PP3/8/PPPN1PPP/R1BQ...,+50
6,rnbqkb1r/ppp2ppp/4pn2/3pP3/3P4/8/PPPN1PPP/R1BQ...,+10
7,rnbqkb1r/pppn1ppp/4p3/3pP3/3P4/8/PPPN1PPP/R1BQ...,+75
8,rnbqkb1r/pppn1ppp/4p3/3pP3/3P1P2/8/PPPN2PP/R1B...,+52
9,rnbqkb1r/pp1n1ppp/4p3/2ppP3/3P1P2/8/PPPN2PP/R1...,+52


In [23]:
#Initiating our FEN to input Vector Function, which will output a (64,1) vector of our chess position.

def FENtoVEC (FEN):
    pieces = {"r":5,"n":3,"b":3.5,"q":9.5,"k":20,"p":1,"R":-5,"N":-3,"B":-3.5,"Q":-9.5,"K":-20,"P":-1}
    FEN = list(str(FEN.split()[0]))
    VEC = []
    for i in range(len(FEN)):
        if FEN[i] == "/":
            continue
        if FEN[i] in pieces:
            VEC.append(pieces[FEN[i]])
        else:
            em = [VEC.append(0) for i in range(int(FEN[i]))]

    return VEC




#Preparing traing data
df = df.head(400000)
board = df["FEN"].tolist()
evl = np.asarray([float(i.strip("#+")) for i in df["Evaluation"].tolist()])
RFevl = np.asarray([float(i.strip("#+")) for i in df["Evaluation"].tolist()])
for i in range(len(evl)):
    if evl[i] > 1000:
        evl[i] = float(1000)
    if evl.item(i) < -1000:
        evl[i] = float(1000)
    evl[i] = evl[i]/1000
NNtrainX = np.asarray([FENtoVEC(state) for state in board])
NNtrainy = evl


# Preparing Validation Data
Edf = Edf.head(10000)
Eboard = Edf["FEN"].tolist()
Eevl = np.asarray([float(i.strip("#+")) for i in Edf["Evaluation"].tolist()])
RFEevl = np.asarray([float(i.strip("#+")) for i in Edf["Evaluation"].tolist()])
for i in range(len(Eevl)):
    if Eevl[i] > 1000:
        Eevl[i] = float(1000)
    if Eevl.item(i) < -1000:
        Eevl[i] = float(1000)
    Eevl[i] = Eevl[i]/1000
NNvalX = np.asarray([FENtoVEC(state) for state in Eboard])
NNvaly = Eevl
RRvalX = pd.DataFrame(NNvalX[:10000])
RRvaly = RFEevl

# Assuming each sample in NNtrainX is a 1D array of length 64
NNtrainX = NNtrainX.reshape(-1, 8, 8, 1)  # Reshape to (num_samples, 8, 8, 1)

# Do the same for the validation data
NNvalX = NNvalX.reshape(-1, 8, 8, 1)  # Reshape to (num_samples, 8, 8, 1)


In [24]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, Flatten

model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(8, 8, 1)))
model.add(Flatten())
model.add(Dense(64, input_dim=64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='tanh'))
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_squared_error'],steps_per_execution = 32)

model.fit(NNtrainX, NNtrainy, epochs=10000,batch_size = 1000, verbose = 0.1, callbacks=[keras.callbacks.EarlyStopping(monitor='loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto'), model_checkpoint])

Epoch 1/10000
Epoch 2/10000
Epoch 3/10000
Epoch 4/10000
Epoch 5/10000
Epoch 6/10000
Epoch 7/10000
Epoch 8/10000
Epoch 9/10000
Epoch 10/10000
Epoch 11/10000
Epoch 12/10000
Epoch 13/10000
Epoch 14/10000
Epoch 15/10000
Epoch 16/10000
Epoch 17/10000
Epoch 18/10000
Epoch 19/10000
Epoch 20/10000
Epoch 21/10000
Epoch 22/10000
Epoch 23/10000
Epoch 24/10000
Epoch 25/10000
Epoch 26/10000
Epoch 27/10000
Epoch 28/10000
Epoch 29/10000
Epoch 30/10000
Epoch 31/10000
Epoch 32/10000
Epoch 33/10000
Epoch 34/10000
Epoch 35/10000
Epoch 36/10000
Epoch 37/10000
Epoch 38/10000
Epoch 39/10000
Epoch 40/10000
Epoch 41/10000
Epoch 42/10000
Epoch 43/10000
Epoch 44/10000
Epoch 45/10000
Epoch 46/10000
Epoch 47/10000
Epoch 48/10000
Epoch 49/10000
Epoch 50/10000
Epoch 51/10000
Epoch 52/10000
Epoch 53/10000
Epoch 54/10000
Epoch 55/10000
Epoch 56/10000
Epoch 57/10000
Epoch 58/10000
Epoch 59/10000
Epoch 60/10000
Epoch 61/10000
Epoch 62/10000
Epoch 63/10000
Epoch 64/10000
Epoch 65/10000
Epoch 66/10000
Epoch 67/10000
Epoc

In [25]:
RFtrain_X = pd.DataFrame(NNtrainX[:10000])
RFtrain_y = RFevl[:10000]
forest_model = RandomForestRegressor(n_estimators=200, max_depth= 32,random_state=1)
forest_model.fit(RFtrain_X, RFtrain_y)

ValueError: Must pass 2-d input. shape=(10000, 8, 8, 1)

In [26]:
RRpre = forest_model.predict(RRvalX)
results = model.evaluate(NNvalX,NNvaly, batch_size=1)
print("Average Pawn loss over New dataset on the Neural Network",(results[0]**0.5)*10)
print("Average Pawn loss over New dataset on the Random Forrest",mean_absolute_error(RRvaly, RRpre)/100)
model.save('model')

10000/10000 [==============================] - 1s 118us/step - loss: 0.3804 - mean_squared_error: 0.2571
Average Pawn loss over New dataset on the Neural Network 6.167502000867923
Average Pawn loss over New dataset on the Random Forrest 5.068980851179364


INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [ ]:
FENtoVEC("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")

[5,
 3,
 3.5,
 9.5,
 20,
 3.5,
 3,
 5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -5,
 -3,
 -3.5,
 -9.5,
 -20,
 -3.5,
 -3,
 -5]

In [ ]:
# Convert FEN to vector
fen_vec = [FENtoVEC("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR")]

# Ensure the input is in the correct format
fen_vec = np.asarray(fen_vec)

# Make a prediction
prediction = model.predict(fen_vec)
prediction2 = forest_model.predict(fen_vec)

prediction[0][0]


ValueError: in user code:

    File "/Users/andrew/anaconda3/envs/TensorFlow/lib/python3.11/site-packages/keras/engine/training.py", line 2175, in predict_function  *
        outputs = step_function(self, iterator)
    File "/Users/andrew/anaconda3/envs/TensorFlow/lib/python3.11/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/andrew/anaconda3/envs/TensorFlow/lib/python3.11/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/andrew/anaconda3/envs/TensorFlow/lib/python3.11/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/Users/andrew/anaconda3/envs/TensorFlow/lib/python3.11/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/andrew/anaconda3/e

In [ ]:
import joblib

joblib.dump(forest_model, './forest_model.joblib')

['./forest_model.joblib']